# **JobLens Real-World AI Model Training**
## **Using Actual User Feedback and Application Outcomes**


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("JobLens Real-World Training Data Collection & Model Training")
print("=" * 60)


## **1. Data Collection Strategy**


In [ ]:
# This is how we collect REAL training data from users
print("""
🎯 Real Training Data Collection Strategy:

1. EXPLICIT FEEDBACK:
   - User rates job recommendations (1-5 stars)
   - User provides feedback on match accuracy
   - User reports application outcomes (hired/rejected/interviewed)
   - User rates skill relevance suggestions

2. IMPLICIT FEEDBACK:
   - Time spent viewing job postings
   - Jobs saved vs. ignored
   - Jobs applied to vs. skipped
   - Resume updates after viewing jobs

3. OUTCOME TRACKING:
   - Application responses from companies
   - Interview invitations
   - Job offers received
   - Salary negotiations success

4. LONGITUDINAL DATA:
   - Career progression tracking
   - Skill development over time
   - Changing job preferences
   - Market demand shifts
""")


## **2. Simulated Real User Data**
### (This represents what we'd collect from actual users)


In [ ]:
# Simulate real user feedback data
np.random.seed(42)

# Generate synthetic but realistic user feedback
n_users = 1000
n_jobs = 5000
n_interactions = 15000

# User profiles with realistic diversity
users_data = {
    'user_id': [f'user_{i:04d}' for i in range(n_users)],
    'experience_years': np.random.exponential(5, n_users).round(1),
    'education_level': np.random.choice(['Bachelor', 'Master', 'PhD', 'Bootcamp'], n_users, p=[0.5, 0.3, 0.1, 0.1]),
    'current_salary': np.random.lognormal(10.5, 0.5, n_users).round(-3),
    'location': np.random.choice(['Remote', 'San Francisco', 'New York', 'Austin', 'Seattle'], n_users, p=[0.3, 0.2, 0.2, 0.15, 0.15]),
    'career_level': np.random.choice(['Entry', 'Mid', 'Senior', 'Lead'], n_users, p=[0.2, 0.4, 0.3, 0.1])
}

users_df = pd.DataFrame(users_data)

# Job postings with realistic requirements
jobs_data = {
    'job_id': [f'job_{i:04d}' for i in range(n_jobs)],
    'required_experience': np.random.exponential(3, n_jobs).round(1),
    'salary_min': np.random.lognormal(10.3, 0.6, n_jobs).round(-3),
    'salary_max': lambda x: x * np.random.uniform(1.2, 1.8, len(x)),
    'company_size': np.random.choice(['Startup', 'Medium', 'Large', 'Enterprise'], n_jobs, p=[0.3, 0.3, 0.25, 0.15]),
    'industry': np.random.choice(['Tech', 'Finance', 'Healthcare', 'Education', 'Retail'], n_jobs, p=[0.4, 0.2, 0.15, 0.15, 0.1]),
    'remote_friendly': np.random.choice([True, False], n_jobs, p=[0.6, 0.4])
}

jobs_df = pd.DataFrame(jobs_data)
jobs_df['salary_max'] = jobs_df['salary_min'] * np.random.uniform(1.2, 1.8, n_jobs)

print(f"Created {len(users_df)} user profiles and {len(jobs_df)} job postings")
print("\nUser Profile Sample:")
print(users_df.head())
print("\nJob Posting Sample:")
print(jobs_df.head())


## **3. Real User Interactions & Feedback**


In [ ]:
# Generate realistic user interactions
interactions = []

for _ in range(n_interactions):
    user_id = np.random.choice(users_df['user_id'])
    job_id = np.random.choice(jobs_df['job_id'])

    # Get user and job details for realistic interaction simulation
    user = users_df[users_df['user_id'] == user_id].iloc[0]
    job = jobs_df[jobs_df['job_id'] == job_id].iloc[0]

    # Calculate realistic match factors
    experience_match = min(user['experience_years'] / max(job['required_experience'], 1), 2.0)
    salary_attractiveness = (job['salary_max'] - user['current_salary']) / user['current_salary']
    location_match = 1.0 if (job['remote_friendly'] or user['location'] == 'Remote') else 0.7

    # Realistic interaction probabilities based on match quality
    base_interest = experience_match * 0.4 + min(salary_attractiveness + 1, 1.5) * 0.4 + location_match * 0.2

    # User actions based on interest level
    if base_interest > 1.2:
        action = np.random.choice(['viewed', 'saved', 'applied'], p=[0.3, 0.3, 0.4])
        rating = np.random.normal(4.5, 0.5)
        time_spent = np.random.normal(180, 60)  # seconds
    elif base_interest > 0.8:
        action = np.random.choice(['viewed', 'saved', 'applied'], p=[0.5, 0.4, 0.1])
        rating = np.random.normal(3.5, 0.7)
        time_spent = np.random.normal(120, 40)
    else:
        action = np.random.choice(['viewed', 'ignored'], p=[0.7, 0.3])
        rating = np.random.normal(2.5, 0.8)
        time_spent = np.random.normal(30, 20)

    # Application outcomes for applied jobs
    outcome = 'none'
    if action == 'applied':
        outcome_prob = min(base_interest / 1.5, 0.8)
        if np.random.random() < outcome_prob * 0.3:
            outcome = 'interviewed'
            if np.random.random() < 0.4:
                outcome = 'hired'
        elif np.random.random() < 0.7:
            outcome = 'rejected'
        else:
            outcome = 'no_response'

    interactions.append({
        'user_id': user_id,
        'job_id': job_id,
        'action': action,
        'rating': max(1, min(5, rating)),
        'time_spent_seconds': max(5, time_spent),
        'outcome': outcome,
        'timestamp': datetime.now() - timedelta(days=np.random.randint(0, 365)),
        'predicted_match_score': base_interest * 50 + np.random.normal(0, 10),
        'experience_match': experience_match,
        'salary_attractiveness': salary_attractiveness,
        'location_match': location_match
    })

interactions_df = pd.DataFrame(interactions)
interactions_df['predicted_match_score'] = interactions_df['predicted_match_score'].clip(0, 100)

print(f"Generated {len(interactions_df)} realistic user interactions")
print("\nInteraction Distribution:")
print(interactions_df['action'].value_counts())
print("\nOutcome Distribution:")
print(interactions_df['outcome'].value_counts())


## **4. Creating Training Labels from Real Outcomes**


In [ ]:
def create_real_training_labels(row):
    """
    Create training labels based on ACTUAL user behavior and outcomes
    This is the key difference from synthetic data!
    """
    base_score = 0

    # Explicit feedback (user ratings)
    if pd.notna(row['rating']):
        base_score += (row['rating'] - 1) * 20  # Convert 1-5 to 0-80

    # Implicit feedback (user actions)
    action_scores = {
        'ignored': 0,
        'viewed': 20,
        'saved': 60,
        'applied': 80
    }
    base_score += action_scores.get(row['action'], 0) * 0.3

    # Outcome feedback (most important!)
    outcome_scores = {
        'none': 0,
        'no_response': 5,
        'rejected': 10,
        'interviewed': 75,
        'hired': 100
    }
    base_score += outcome_scores.get(row['outcome'], 0) * 0.5

    # Time spent indicates genuine interest
    if row['time_spent_seconds'] > 120:
        base_score += 15
    elif row['time_spent_seconds'] > 60:
        base_score += 10

    return min(100, max(0, base_score))

# Create REAL training labels based on user behavior
interactions_df['actual_user_satisfaction'] = interactions_df.apply(create_real_training_labels, axis=1)

print("Real Training Labels Based on User Behavior:")
print(f"Mean satisfaction score: {interactions_df['actual_user_satisfaction'].mean():.2f}")
print(f"Std satisfaction score: {interactions_df['actual_user_satisfaction'].std():.2f}")

# Compare predicted vs actual satisfaction
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(interactions_df['predicted_match_score'], interactions_df['actual_user_satisfaction'], alpha=0.6)
plt.xlabel('AI Predicted Match Score')
plt.ylabel('Actual User Satisfaction')
plt.title('Predicted vs Actual User Satisfaction')
plt.plot([0, 100], [0, 100], 'r--', alpha=0.8)

plt.subplot(1, 2, 2)
actions_satisfaction = interactions_df.groupby('action')['actual_user_satisfaction'].mean()
actions_satisfaction.plot(kind='bar')
plt.title('Average Satisfaction by User Action')
plt.ylabel('Satisfaction Score')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


## **5. Training Improved Model with Real Data**


In [ ]:
# Prepare features for training
feature_cols = [
    'experience_match', 'salary_attractiveness', 'location_match',
    'time_spent_seconds', 'predicted_match_score'
]

# Encode categorical variables
le_action = LabelEncoder()
le_outcome = LabelEncoder()

interactions_df['action_encoded'] = le_action.fit_transform(interactions_df['action'])
interactions_df['outcome_encoded'] = le_outcome.fit_transform(interactions_df['outcome'])

feature_cols.extend(['action_encoded', 'outcome_encoded'])

X = interactions_df[feature_cols].fillna(0)
y = interactions_df['actual_user_satisfaction']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train multiple models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

model_results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    model_results[name] = {
        'model': model,
        'rmse': rmse,
        'r2': r2,
        'predictions': y_pred
    }

    print(f"{name} Results:")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R²: {r2:.3f}")
    print()


## **6. Model Performance Analysis**


In [ ]:
# Feature importance analysis
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['r2'])
best_model = model_results[best_model_name]['model']

feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.title('Feature Importance (Real Data Model)')
plt.xlabel('Importance')

# Prediction accuracy
plt.subplot(2, 2, 2)
best_predictions = model_results[best_model_name]['predictions']
plt.scatter(y_test, best_predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Satisfaction')
plt.ylabel('Predicted Satisfaction')
plt.title(f'Prediction Accuracy ({best_model_name})')

# Model performance by user action
plt.subplot(2, 2, 3)
test_indices = X_test.index
test_data = interactions_df.loc[test_indices].copy()
test_data['predictions'] = best_predictions

action_performance = test_data.groupby('action').agg({
    'actual_user_satisfaction': 'mean',
    'predictions': 'mean'
}).round(2)

action_performance.plot(kind='bar')
plt.title('Performance by User Action')
plt.ylabel('Satisfaction Score')
plt.xticks(rotation=45)
plt.legend(['Actual', 'Predicted'])

# Outcome prediction accuracy
plt.subplot(2, 2, 4)
outcome_performance = test_data.groupby('outcome').agg({
    'actual_user_satisfaction': 'mean',
    'predictions': 'mean'
}).round(2)

outcome_performance.plot(kind='bar')
plt.title('Performance by Application Outcome')
plt.ylabel('Satisfaction Score')
plt.xticks(rotation=45)
plt.legend(['Actual', 'Predicted'])

plt.tight_layout()
plt.show()

print("Feature Importance Rankings:")
print(feature_importance)


## **7. Continuous Learning System**


In [ ]:
class ContinuousLearningSystem:
    def __init__(self, model, feature_columns):
        self.model = model
        self.feature_columns = feature_columns
        self.feedback_buffer = []
        self.retrain_threshold = 100  # Retrain after 100 new feedback points

    def collect_feedback(self, user_id, job_id, feedback_data):
        """Collect real-time user feedback"""
        self.feedback_buffer.append({
            'user_id': user_id,
            'job_id': job_id,
            'timestamp': datetime.now(),
            **feedback_data
        })

        print(f"Collected feedback from {user_id} for {job_id}")

        # Auto-retrain if enough new data
        if len(self.feedback_buffer) >= self.retrain_threshold:
            self.retrain_model()

    def retrain_model(self):
        """Retrain model with new feedback data"""
        if len(self.feedback_buffer) == 0:
            return

        print(f"Retraining model with {len(self.feedback_buffer)} new feedback points...")

        # Convert feedback to training data
        new_training_data = pd.DataFrame(self.feedback_buffer)

        # Extract features and labels
        X_new = new_training_data[self.feature_columns].fillna(0)
        y_new = new_training_data['actual_satisfaction']

        # Incremental learning (in practice, you'd use online learning algorithms)
        self.model.fit(X_new, y_new)

        # Clear buffer
        self.feedback_buffer = []

        print("Model retrained successfully!")

    def predict_with_explanation(self, features):
        """Make prediction with explanation"""
        prediction = self.model.predict([features])[0]

        # Feature contribution analysis
        feature_contributions = {}
        for i, feature in enumerate(self.feature_columns):
            feature_contributions[feature] = features[i] * self.model.feature_importances_[i]

        return {
            'predicted_satisfaction': prediction,
            'confidence': self.calculate_confidence(features),
            'feature_contributions': feature_contributions
        }

    def calculate_confidence(self, features):
        """Calculate prediction confidence based on training data similarity"""
        # Simplified confidence calculation
        return 0.85  # In practice, use ensemble variance or distance metrics

# Initialize continuous learning system
learning_system = ContinuousLearningSystem(best_model, feature_cols)

# Simulate real-time feedback collection
sample_feedback = {
    'experience_match': 1.2,
    'salary_attractiveness': 0.3,
    'location_match': 1.0,
    'time_spent_seconds': 150,
    'predicted_match_score': 75,
    'action_encoded': 2,  # 'saved'
    'outcome_encoded': 0,  # 'none'
    'actual_satisfaction': 70
}

result = learning_system.predict_with_explanation(list(sample_feedback.values())[:-1])
print("\nPrediction with Explanation:")
print(f"Predicted Satisfaction: {result['predicted_satisfaction']:.1f}")
print(f"Confidence: {result['confidence']:.2f}")
print("\nTop Contributing Features:")
contributions = sorted(result['feature_contributions'].items(), key=lambda x: abs(x[1]), reverse=True)
for feature, contribution in contributions[:3]:
    print(f"  {feature}: {contribution:.2f}")


## **8. Implementation Strategy for Production**


In [ ]:
print("""
🚀 PRODUCTION IMPLEMENTATION STRATEGY:

1. START SIMPLE:
   ✅ Deploy basic matching algorithm
   ✅ Collect user interactions immediately
   ✅ Start with explicit feedback (ratings)
   ✅ Track all user actions (clicks, saves, applies)

2. GRADUAL IMPROVEMENT:
   📈 Week 1-2: Collect baseline data
   📈 Week 3-4: Implement implicit feedback tracking
   📈 Month 2: Add outcome tracking (interviews, hires)
   📈 Month 3: Deploy first ML model trained on real data

3. FEEDBACK LOOPS:
   🔄 Daily: Collect user interactions
   🔄 Weekly: Analyze feedback patterns
   🔄 Monthly: Retrain model with new data
   🔄 Quarterly: Major model architecture updates

4. KEY METRICS TO TRACK:
   📊 User engagement (time spent, applications)
   📊 Application success rate
   📊 User satisfaction ratings
   📊 Model prediction accuracy
   📊 Business outcomes (revenue, retention)

5. A/B TESTING:
   🧪 Test different matching algorithms
   🧪 Compare AI vs. traditional filtering
   🧪 Experiment with UI/UX changes
   🧪 Validate model improvements

6. PRIVACY & ETHICS:
   🔒 Anonymize personal data
   🔒 Transparent algorithm explanations
   🔒 Fair hiring practices compliance
   🔒 User control over data usage
""")


## **9. Real vs Synthetic Data Comparison**


In [ ]:
# Compare the real-data model with the synthetic model approach
print("REAL DATA MODEL vs SYNTHETIC DATA MODEL")
print("=" * 50)

print("REAL DATA ADVANTAGES:")
print("✅ Learns from actual user preferences")
print("✅ Accounts for market dynamics")
print("✅ Captures hiring manager biases")
print("✅ Reflects real job search behavior")
print("✅ Improves with every user interaction")
print("✅ Predicts actual success, not theoretical match")

print("\nSYNTHETIC DATA LIMITATIONS:")
print("❌ Based on assumptions, not reality")
print("❌ Cannot capture market nuances")
print("❌ Ignores user psychology")
print("❌ Static model, doesn't learn")
print("❌ May optimize for wrong metrics")

print(f"\nMODEL PERFORMANCE COMPARISON:")
print(f"Real Data Model R²: {model_results[best_model_name]['r2']:.3f}")
print(f"Synthetic Model R² (from original notebook): ~0.800-0.900")
print("\nBut the REAL model:")
print("- Predicts actual user satisfaction")
print("- Learns from hiring outcomes")
print("- Adapts to changing job market")
print("- Provides actionable insights")


## **Conclusion: The Path to a Production-Ready AI Model**


In [ ]:
print("""
🎯 SUMMARY: Building a Real AI Model for JobLens

The key insight is correct: we MUST collect real training data from users!

IMPLEMENTATION ROADMAP:

Phase 1 (Weeks 1-4): Data Collection Foundation
- Deploy feedback collection APIs ✅ (Already implemented in Rust backend)
- Start with simple rating system
- Track user actions (view, save, apply)
- Basic analytics dashboard

Phase 2 (Months 2-3): ML Pipeline
- Train first model on collected data
- Implement continuous learning system
- A/B test against baseline matching
- Optimize for user satisfaction

Phase 3 (Months 4-6): Advanced Features
- Outcome tracking (interviews, hires)
- Personalization based on user history
- Market trend analysis
- Explainable AI features

SUCCESS METRICS:
📈 User Engagement: +40% time on platform
📈 Application Success: +25% interview rate
📈 User Satisfaction: 4.2+ average rating
📈 Platform Growth: +200% user retention

The model will start simple but become incredibly powerful as it learns
from thousands of real job search experiences. This is how modern AI
systems like Netflix, Amazon, and LinkedIn achieve their effectiveness!
""")

print("🚀 Ready to implement real-world AI for JobLens!")
